高斯混合模型是一种概率模型，`假设所有数据点都是从有限数量的高斯分布的混合参数中生成的`

`基于分布`的模型,高斯混合模型倾向于**将属于单一分布的数据点组合在一起**。

可以将混合模型视为对 k-means聚类算法的扩展，它包含了数据的协方差结构以及隐高斯模型中心的信息。

https://www.analyticsvidhya.com/blog/2019/10/gaussian-mixture-models-clustering/

## 高斯混合 -- Gaussian Mixture

GMM将确定每个数据点属于这些分布的概率（软聚类）。比如下图中，红色点属于蓝色的概率为0.2，属于青色的概率为0.8。

![20220715223000](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715223000.png)

多维（二维）高斯分布：（下图是二维高斯分布，其中，均值变成2维，方差变成2*2的协方差）

![20220715224101](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715224101.png)

### 期望最大化算法 -- Expectation-Maximization(EM)

一般情况下，`从样本观察数据中，找出样本的模型参数。最常用的方法就是极大对数似然`。但在一些情况下，得到的观察数据有未观察到的隐含数据，此时我们未知的有隐含数据和模型参数，因而无法直接用极大对数似然函数得到模型分布的参数。

![20220716094722](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220716094722.png)

如果从算法思想的角度来思考EM算法，可以发现我们的算法里已知的是观察数据，未知的是隐含数据和模型参数，
 - 在E步，我们所做的事情是固定模型参数的值，优化隐含数据的分布，
 - 在M步，我们所做的事情是固定隐含数据分布，优化模型参数的值。

https://www.cnblogs.com/pinard/p/6912636.html

以上面求解二维高斯分布为例，数据集是d维的，高斯分布分为k簇，每一簇都有其均值与协方差，该如何求解高斯分布的参数呢？

一般情况下，如果我知道每个数据点属于哪个簇，我就可以通过簇内的数据估计出均值、方差，但目前并不知道每个数据点归属于哪个分布（这也叫潜在变量，latent variable）。由于我没有潜在变量的值，期望最大化算法试图使用现有数据来确定这些变量的最佳值，然后找到模型参数。基于这些模型参数，我们再去更新潜变量的值，如此反复。

所以，上面问题通过EM算法分2步：

1. 利用现有的数据估计（猜测）潜在变量的值，在这个例子指的就是数据属于哪个簇的？
2. 基于第1步产生的估计值，然后就可以用所有数据去更新参数（参数一开始是随机初始化的）；

试着用EM算法求解以上问题：

假设我们需要分配k个聚类，这意味着有k个高斯分布，其平均值、协方差值分别是μ1, μ2, ... μk和Σ1, Σ2, ... Σk。此外，分布还有一个参数，分布的密度用Πi表示。现在已经确定了聚类的数量k，并随机分配了均值、协方差和密度的值。现在开始套用EM的逻辑：

1. 对于每个点xi，计算它属于聚类c1, c2, ... ck的概率：

    ![20220715234223](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715234223.png)
    
2. 密度，新的密度是由簇内点的数量和总的点的数量的比率来定义的：
    
    ![20220715234419](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715234419.png)

3. 通过极大似然求解均值、协方差 ，均值（求期望并进行标准化）和协方差矩阵是根据分配给分布的值更新的，与数据点的概率值成比例：
   
   ![20220715234546](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715234546.png)

4. 根据更新均值、协方差后的高斯分布，重新估计每个点属于不同聚类的概率，重复2、3步；

**k-means只考虑平均值来更新中心点，而GMM则考虑到了数据的平均值与方差**。
   
   
   




`GaussianMixture` 对象实现了用来拟合高斯混合模型的期望最大化 (EM) 算法。它还可以为多变量模型`绘制置信椭圆体`，同时`计算 BIC（Bayesian Information Criterion，贝叶斯信息准则）来评估数据中聚类的数量`。

GaussianMixture.fit 方法可以从训练数据中拟合出一个高斯混合模型。如果给定测试数据，通过使用 GaussianMixture.predict 方法，可以为每个样本分配最适合的高斯分布模型。

GaussianMixture 方法中自带了不同的选项来约束不同估类的协方差：spherical，diagonal，tied 或 full 协方差

![20220715114450](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715114450.png)

一般在进行高斯混合聚类前，需要通过K-means等方式初始化几个中心点，并为每个点分别计算由该混合模型内的每个分量生成的概率。然后，调整模型参数以最大化模型生成这些参数的可能性。重复这个过程（EM），该算法保证该过程内的参数总会收敛到一个局部最优解。

![20220715115601](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220715115601.png)



In [1]:
import numpy as np
from sklearn.mixture import GaussianMixture

X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])

# fit: EM算法估计模型参数
gm = GaussianMixture(n_components=2, random_state=0,init_params='kmeans').fit(X)
gm.means_

array([[10.,  2.],
       [ 1.,  2.]])

In [2]:
# 预测所属聚类类别
print(gm.predict([[0, 0], [12, 3]]))

# 聚类后给出聚类标签
gm.fit_predict(X)

[1 0]


array([1, 1, 1, 0, 0, 0], dtype=int64)

## 变分高斯混合 -- Variational Bayesian Gaussian Mixture

`变分推断`是一种估计算法，`使模型证据（包括先验）的下限最大化，而不是数据似然`。

变分推断的原理与期望最大化相同（即两者都是迭代算法，在寻找每个混合分布产生的每个点的概率和将混合分布拟合到这些指定的点之间交替进行），但`变分方法通过整合先验分布的信息来增加正则化`,避免期望最大化中经常出现的奇异现象，但也给模型引入了一些微妙的偏差。推理的速度通常明显较慢。

In [3]:
import numpy as np
from sklearn.mixture import BayesianGaussianMixture

X = np.array([[1, 2], [1, 4], [1, 0], [4, 2], [12, 4], [10, 7]])
bgm = BayesianGaussianMixture(n_components=2, random_state=42).fit(X)
bgm.means_

bgm.predict([[0, 0], [9, 3]])

array([0, 1], dtype=int64)